# 🚀 ![Fundamentals](https://img.shields.io/badge/Fundamentals-4CAF50) Quickstart


{{ badge }}

A fast, example-driven tour of Autoform. This notebook covers the core workflow: trace a function, transform the IR, and execute it.

Topics covered:

- Tracing functions into IR with `trace`
- Executing IR with `ir.call()` / `ir.acall()`
- Debugging with `checkpoint` + `collect`
- Vectorization with `batch`
- Semantic gradients with `pullback`
- Concurrent execution with `sched`

In [1]:
!uv pip install --quiet "git+https://github.com/ASEM000/autoform.git"

## 0) Setup

Key concepts:
- Autoform uses LiteLLM as backend
- Any LiteLLM-supported provider works https://docs.litellm.ai/docs/providers


In [1]:
import getpass
import os
import time
import warnings

import autoform as af

warnings.filterwarnings("ignore", category=UserWarning)

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY: ")

MODEL = "openai/gpt-4o"

## 1) Trace with `trace`

Key concepts:
- `trace(func)(examples)` captures a function as IR
- IR is a data structure representing the computation
- Example inputs define structure; values replaced at call time


In [2]:
def greet(name: str) -> str:
    return af.format("Hello, {}!", name)


# NOTE: "..." is a placeholder; the actual value is passed at call time
ir_greet = af.trace(greet)("...")
print(ir_greet)

func(%0:IRVar[str]) -> (%1:IRVar[str]) {
  (%1:IRVar[str]) = format(%0:IRVar[str], template='Hello, {}!', keys=())
}


## 2) Execute with `ir.call()`

Key concepts:
- `ir.call(inputs)` runs synchronously
- `ir.acall(inputs)` runs asynchronously
- Inputs replace placeholders from tracing


In [3]:
result = ir_greet.call("Alice")
print(result)

Hello, Alice!


## 3) Structured LLM calls with `struct_lm_call`

Key concepts:
- `struct_lm_call(messages, model=..., struct=...)` returns typed output
- Schema enforced at LLM level, no parsing needed
- `Struct` fields are differentiable


In [5]:
from typing import Literal


class Answer(af.Struct):
    value: str
    confidence: Literal["low", "medium", "high"]


def ask(question: str) -> Answer:
    msg = dict(role="user", content=question)
    return af.struct_lm_call([msg], model=MODEL, struct=Answer)


ir_ask = af.trace(ask)("...")
print(ir_ask.call("What is 2+2?"))

value='4' confidence='high'


## 4) Debug with `checkpoint` and `collect`

Key concepts:
- `checkpoint(value, key=...)` marks a value for capture
- `collect(collection=...)` returns captured values
- Non-invasive: original computation unchanged


In [4]:
def pipeline(x: str) -> str:
    step1 = af.format("Step 1: {}", x)
    step1 = af.checkpoint(step1, key="after_step1", collection="debug")
    step2 = af.format("Step 2: {}", step1)
    return step2


ir_pipe = af.trace(pipeline)("...")

# NOTE: collect is a context manager, not a function call
with af.collect(collection="debug") as collected:
    result = ir_pipe.call("input")

print(f"Result: {result}")
print(f"Captured: {collected}")

Result: Step 2: Step 1: input
Captured: defaultdict(<class 'list'>, {'after_step1': ['Step 1: input']})


## 5) Vectorize with `batch`

Key concepts:
- `batch(ir, in_axes=...)` creates vectorized IR
- `True` for batched inputs, `False` for broadcast
- Similar to `jax.vmap` for text operations


In [5]:
def rewrite(style: str, text: str) -> str:
    return af.format("[{}] {}", style, text)


ir_rewrite = af.trace(rewrite)("...", "...")

# NOTE: style is broadcast (False), texts are batched (True)
ir_batched = af.batch(ir_rewrite, in_axes=(False, True))

texts = ["hello world", "goodbye moon", "good morning sun"]
results = ir_batched.call(("UPPERCASE", texts))
print(results)

['[UPPERCASE] hello world', '[UPPERCASE] goodbye moon', '[UPPERCASE] good morning sun']


## 6) Semantic gradients with `pullback`

Key concepts:
- `pullback(ir)` creates a gradient IR
- Input: `((inputs), feedback)`, Output: `(output, grad_inputs)`
- Foundation for prompt optimization


In [ ]:
def summarize(text: str) -> str:
    msg = dict(role="user", content=af.format("Summarize: {}", text))
    return af.lm_call([msg], model=MODEL)


ir_sum = af.trace(summarize)("...")
ir_pb = af.pullback(ir_sum)

text = "Autoform is a library for composable LLM program transformations."
feedback = "Too short. Include more detail about transformations."

# NOTE: pullback returns (output, grad_tree) where grad_tree matches input structure
output, grad_text = ir_pb.call((text, feedback))
print(f"Output: {output}")
print(f"Gradient: {grad_text}")

Output: Autoform is a library designed to facilitate the creation and management of composable transformations in large language model (LLM) programs. It provides a framework that allows developers to efficiently build and manipulate LLM-based applications by composing various program transformations. This can streamline the development process and enhance the capabilities of LLMs through modular and reusable components.
Gradient: To address the feedback and improve the OUTPUT by including more detail about transformations, adjust the INPUT as follows:

1. **Specify Types of Transformations**: Include information on the types of transformations Autoform supports (e.g., syntax transformations, semantic transformations, or optimization transformations).

2. **Describe Key Features**: Highlight key features of Autoform such as modularity, reusability, and ease of integration with existing LLM systems.

3. **Provide Use Cases**: Incorporate examples or use cases of how Autoform can be appl

## 7) Parallelize with `sched`

Key concepts:
- `sched(ir)` rewrites IR for parallel execution
- Use with `ir.acall()` for concurrency
- Write sequential code, run concurrent

Compare sequential vs scheduled execution time:


In [9]:
def parallel_calls(doc: str) -> str:
    doc = af.stop_gradient(doc)
    # NOTE: These two calls are independent
    msg1 = dict(role="user", content=af.format("Summarize in 1 sentence: {}", doc))
    summary = af.lm_call([msg1], model=MODEL)
    msg2 = dict(role="user", content=af.format("List 2 topics: {}", doc))
    topics = af.lm_call([msg2], model=MODEL)
    return af.format("Summary: {}\nTopics: {}", summary, topics)


ir_calls = af.trace(parallel_calls)("...")
ir_scheduled = af.sched(ir_calls)

doc = "Machine learning models require training data and compute resources."

# Sequential: calls run one after another
t1 = time.time()
result_seq = ir_calls.call(doc)
t_seq = time.time() - t1

# Scheduled: independent calls run concurrently
t2 = time.time()
result_sched = await ir_scheduled.acall(doc)
t_sched = time.time() - t2

print(f"Sequential: {t_seq:.2f}s")
print(f"Scheduled:  {t_sched:.2f}s")
print(f"Speedup:    {t_seq / t_sched:.1f}x")

Sequential: 4.58s
Scheduled:  1.48s
Speedup:    3.1x
